<a href="https://colab.research.google.com/github/GGlivePh/PLF/blob/main/swe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
!git clone https://github.com/GGlivePh/PLF.git

Cloning into 'PLF'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 21 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 119.55 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [92]:
import os
import xml
import qrcode
import xml.etree.ElementTree as ET
import io
import base64
from joblib import Parallel, delayed
import subprocess

os.environ['MAGICK_MEMORY_LIMIT'] = '28GB' # increase to a higher value if needed
os.environ['MAGICK_DISK_LIMIT'] = '28GB' # increase to a higher value if needed
DATE="2024-11-26"


n_labs = 24
START= 0

configs ={
    "large":{
        "N_REPS": 2,
        "n_blanks": 0,
        "OUT_DIR": "labels-large",
        "TEMPLATE": "template_large.svg",
        "TILES":  "4x12"
    },


    "small":{
        "N_REPS":3,
        "n_blanks" : 8,
        "OUT_DIR": "labels",
        "TEMPLATE": "template.svg",
        "TILES":  "5x16"}
}


def convert(output):
    png = os.path.splitext(output)[0] + ".png"
    subprocess.run(["convert",  "-density", "1200",output,  png])
    assert os.path.isfile(png)


def make_one_lab(args):
    i,conf = args

    lab = f"{DATE}_{i:04d}"
    if i >= START + n_labs:
        root[2][0][0].text = "DUMMY!"
        img = qrcode.make(lab)
        n_rep = 1
    else:
        root[2][0][0].text = lab
        img = qrcode.make(lab)
        n_rep = conf["N_REPS"]

    with io.BytesIO() as output:
        img.save(output, format="PNG")
        content = base64.b64encode(output.getbuffer())

    new_image = b"data:image/png;base64," + content
    root[2][2].attrib["{http://www.w3.org/1999/xlink}href"] = new_image.decode()

    for rep in range(n_rep):
        output = f"{conf['OUT_DIR']}/{lab}-{rep}.svg"
        with open(output, "wb") as f:
            tree.write(f)
            convert(output)

for mode,conf in configs.items():
    os.makedirs(conf["OUT_DIR"],exist_ok=True)
    tree = ET.parse(conf["TEMPLATE"])
    root = tree.getroot()

    Parallel(n_jobs=1, prefer="processes")(
        delayed(make_one_lab)((i, conf))  for i in range(START, START+n_labs + conf["n_blanks"])
    )
    tmp_file = "/tmp/all_labels.png"
    command = f'montage {conf["OUT_DIR"]}/{DATE}*.png -tile {conf["TILES"]} -geometry +0+0 {tmp_file}'
    print(command)
    os.system(command)
    command = f'mogrify -gravity center -extent  9763x14009 {tmp_file}'
    print(command)
    os.system(command)
    command = f'convert {tmp_file} -page a4 {DATE}_{mode}_labels.pdf'
    print(command)
    os.system(command)

    !for i in *.svg; do convert   -density 1200 $i  $(echo $i | sed s/svg/png/g); done

    !montage  2024-*.png  -tile 4x12   -geometry +0+0 all_labs.png
    !mogrify -gravity center -extent  9763x14009 all_labs.png && convert all_labs.png  -page a4 all_labs.pdf
    # #

montage labels-large/2024-11-26*.png -tile 4x12 -geometry +0+0 /tmp/all_labels.png
mogrify -gravity center -extent  9763x14009 /tmp/all_labels.png
convert /tmp/all_labels.png -page a4 2024-11-26_large_labels.pdf
mogrify-im6.q16: cache resources exhausted `all_labs.png' @ error/cache.c/OpenPixelCache/4095.
montage labels/2024-11-26*.png -tile 5x16 -geometry +0+0 /tmp/all_labels.png
mogrify -gravity center -extent  9763x14009 /tmp/all_labels.png
convert /tmp/all_labels.png -page a4 2024-11-26_small_labels.pdf
mogrify-im6.q16: cache resources exhausted `all_labs.png' @ error/cache.c/OpenPixelCache/4095.


In [97]:
!whereis magick

magick:


In [101]:
import os
import xml
import qrcode
import xml.etree.ElementTree as ET
import io
import base64
from joblib import Parallel, delayed
import subprocess


DATE = "2024-11-26"
n_labs = 24
START = 1

configs = {
    "large": {
        "N_REPS": 2,
        "n_blanks": 0,
        "OUT_DIR": "labels-large",
        "TEMPLATE": "template_large.svg",
        "TILES": "4x12",
    },
    "small": {
        "N_REPS": 3,
        "n_blanks": 8,
        "OUT_DIR": "labels",
        "TEMPLATE": "template.svg",
        "TILES": "5x16",
    },
}


def convert(output):
    png = os.path.splitext(output)[0] + ".png"
    # Use 'convert' instead of 'magick'
    subprocess.run(["convert", "-density", "1200", output, png])
    assert os.path.isfile(png)


def make_one_lab(args):
    i, conf = args

    lab = f"{DATE}_{i:04d}"
    if i >= START + n_labs:
        root[2][0][0].text = "DUMMY!"
        img = qrcode.make(lab)
        n_rep = 1
    else:
        root[2][0][0].text = lab
        img = qrcode.make(lab)
        n_rep = conf["N_REPS"]

    with io.BytesIO() as output:
        img.save(output, format="PNG")
        content = base64.b64encode(output.getbuffer())

    new_image = b"data:image/png;base64," + content
    root[2][2].attrib["{http://www.w3.org/1999/xlink}href"] = new_image.decode()

    for rep in range(n_rep):
        output = f"{conf['OUT_DIR']}/{lab}-{rep}.svg"
        with open(output, "wb") as f:
            tree.write(f)
            convert(output)

for mode, conf in configs.items():
    os.makedirs(conf["OUT_DIR"], exist_ok=True)
    tree = ET.parse(conf["TEMPLATE"])
    root = tree.getroot()

    Parallel(n_jobs=1, prefer="processes")(
        delayed(make_one_lab)((i, conf))
        for i in range(START, START + n_labs + conf["n_blanks"])
    )
    tmp_file = "/tmp/all_labels.png"

    # Set ImageMagick environment variables
    os.environ['MAGICK_MEMORY_LIMIT'] = '28GB'
    os.environ['MAGICK_DISK_LIMIT'] = '28GB'


    command = (
        f'montage {conf["OUT_DIR"]}/{DATE}*.png -tile {conf["TILES"]} '
        f'-geometry +0+0 {tmp_file}'
    )
    print(command)
    # Update this line to use 'convert' for montage as well
    os.system(command.replace("magick", "convert"))

    command = f'mogrify -gravity center -extent  9763x14009 {tmp_file}'
    print(command)
    # Update this line to use 'convert' for mogrify as well
    os.system(command.replace("magick", "convert"))

    command = f'magick {tmp_file} -page a4 {DATE}_{mode}_labels.pdf'
    print(command)
    # Update this line to use 'convert' for the final conversion to PDF


montage labels-large/2024-11-26*.png -tile 4x12 -geometry +0+0 /tmp/all_labels.png
mogrify -gravity center -extent  9763x14009 /tmp/all_labels.png
magick /tmp/all_labels.png -page a4 2024-11-26_large_labels.pdf
montage labels/2024-11-26*.png -tile 5x16 -geometry +0+0 /tmp/all_labels.png
mogrify -gravity center -extent  9763x14009 /tmp/all_labels.png
magick /tmp/all_labels.png -page a4 2024-11-26_small_labels.pdf


In [109]:
import os
print(os.getcwd()) # Print the current working directory
#os.chdir('PLF')  # Change to the correct directory

/content/QG/SelRas/PLF


In [68]:
!git clone https://github.com/GGlivePh/PLF.git

Cloning into 'PLF'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 21 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 119.55 KiB | 1.16 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [69]:
import os
os.chdir('PLF')

In [75]:
!apt-get update
!apt-get install imagemagick -y

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 2s (180 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [81]:
!apt-get update
!apt-get remove imagemagick
!apt-get install imagemagick -y
!which magick

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [88]:
import os
import xml
import qrcode
import xml.etree.ElementTree as ET
import io
import base64
from joblib import Parallel, delayed
import subprocess


DATE="2024-11-26"


n_labs = 24
START= 0

configs ={
    "large":{
        "N_REPS": 2,
        "n_blanks": 0,
        "OUT_DIR": "labels-large",
        "TEMPLATE": "template_large.svg",
        "TILES":  "4x12"
    },


    "small":{
        "N_REPS":3,
        "n_blanks" : 8,
        "OUT_DIR": "labels",
        "TEMPLATE": "template.svg",
        "TILES":  "5x16"}
}


def convert(output):
    png = os.path.splitext(output)[0] + ".png"
    subprocess.run(["convert",  "-density", "1200",output,  png])
    assert os.path.isfile(png)


def make_one_lab(args):
    i,conf = args

    lab = f"{DATE}_{i:04d}"
    if i >= START + n_labs:
        root[2][0][0].text = "DUMMY!"
        img = qrcode.make(lab)
        n_rep = 1
    else:
        root[2][0][0].text = lab
        img = qrcode.make(lab)
        n_rep = conf["N_REPS"]

    with io.BytesIO() as output:
        img.save(output, format="PNG")
        content = base64.b64encode(output.getbuffer())

    new_image = b"data:image/png;base64," + content
    root[2][2].attrib["{http://www.w3.org/1999/xlink}href"] = new_image.decode()

    for rep in range(n_rep):
        output = f"{conf['OUT_DIR']}/{lab}-{rep}.svg"
        with open(output, "wb") as f:
            tree.write(f)
            convert(output)

for mode,conf in configs.items():
    os.makedirs(conf["OUT_DIR"],exist_ok=True)
    tree = ET.parse(conf["TEMPLATE"])
    root = tree.getroot()

    Parallel(n_jobs=1, prefer="processes")(
        delayed(make_one_lab)((i, conf))  for i in range(START, START+n_labs + conf["n_blanks"])
    )
    tmp_file = "/tmp/all_labels.png"
    command = f'montage {conf["OUT_DIR"]}/{DATE}*.png -tile {conf["TILES"]} -geometry +0+0 {tmp_file}'
    print(command)
    os.system(command)
    command = f'mogrify -gravity center -extent  9763x14009 {tmp_file}'
    print(command)
    os.system(command)
    command = f'convert {tmp_file} -page a4 {DATE}_{mode}_labels.pdf'
    print(command)
    os.system(command)

    !for i in *.svg; do convert   -density 1200 $i  $(echo $i | sed s/svg/png/g); done

    !montage  2024-*.png  -tile 4x12   -geometry +0+0 all_labs.png
    !mogrify -gravity center -extent  9763x14009 all_labs.png && convert all_labs.png  -page a4 all_labs.pdf
    # #

montage labels-large/2024-11-26*.png -tile 4x12 -geometry +0+0 /tmp/all_labels.png
mogrify -gravity center -extent  9763x14009 /tmp/all_labels.png
convert /tmp/all_labels.png -page a4 2024-11-26_large_labels.pdf
mogrify-im6.q16: cache resources exhausted `all_labs.png' @ error/cache.c/OpenPixelCache/4095.
montage labels/2024-11-26*.png -tile 5x16 -geometry +0+0 /tmp/all_labels.png
mogrify -gravity center -extent  9763x14009 /tmp/all_labels.png
convert /tmp/all_labels.png -page a4 2024-11-26_small_labels.pdf
mogrify-im6.q16: cache resources exhausted `all_labs.png' @ error/cache.c/OpenPixelCache/4095.
